# 🎨 SDXL Vintage Illustration Notebook — PRO

Stable Diffusion XL 1.0 preconfigured for **vintage ink+watercolor** illustrations (old book style, retro cartoon aesthetic).

### Included
- **Mode Selector** (CPU / GPU basic / GPU optimal)
- **Text2Img** (default vintage style + custom prompt)
- **Img2Img** (photo → vintage illustration)
- **ControlNet (Canny)** for pose/contour consistency
- **Upscale** (x4) for higher resolution
- **Color tone control** (dropdown + custom)
- Saving outputs to `/content/outputs` with timestamps


In [ ]:
# 🔧 Mode Selector (set once, can be changed any time)
MODE = "GPU_OPTIMAL"  # options: "CPU", "GPU_BASIC", "GPU_OPTIMAL"
print(f"Selected MODE: {MODE}")

In [ ]:
# 📦 Install dependencies
!pip -q install diffusers==0.29.0 transformers accelerate xformers safetensors opencv-python-headless ipywidgets
import os, torch
os.environ['PYTORCH_ENABLE_MPS_FALLBACK'] = '1'
print("Dependencies installed.")

In [ ]:
# 🔁 Imports & utils
import os, gc, time, random, datetime
from PIL import Image
import numpy as np
import cv2
import torch
from diffusers import (
    DiffusionPipeline,
    StableDiffusionXLPipeline,
    StableDiffusionXLImg2ImgPipeline,
    StableDiffusionUpscalePipeline,
    ControlNetModel,
    StableDiffusionXLControlNetPipeline,
)

DEVICE = 'cuda' if torch.cuda.is_available() and MODE != 'CPU' else 'cpu'
DTYPE = torch.float16 if DEVICE == 'cuda' else torch.float32
print(f"DEVICE: {DEVICE}, DTYPE: {DTYPE}")

OUTDIR = "/content/outputs"
os.makedirs(OUTDIR, exist_ok=True)

def save_image(img: Image.Image, prefix: str = "image"):
    ts = datetime.datetime.now().strftime('%Y%m%d_%H%M%S')
    path = os.path.join(OUTDIR, f"{prefix}_{ts}.png")
    img.save(path)
    print("Saved:", path)
    return path

def set_seed(seed: int | None):
    if seed is None:
        return None
    g = torch.Generator(device=DEVICE)
    g.manual_seed(seed)
    return g

def free_memory():
    gc.collect()
    if torch.cuda.is_available():
        torch.cuda.empty_cache()
    print("Memory cleared.")

In [ ]:
# 🧠 Model loaders (lazy)
TXT2IMG_MODEL_ID = "stabilityai/stable-diffusion-xl-base-1.0"
UPSCALE_MODEL_ID = "stabilityai/stable-diffusion-x4-upscaler"
CONTROLNET_CANNY_ID = "diffusers/controlnet-canny-sdxl-1.0"  # SDXL canny ControlNet

_txt2img_pipe = None
_img2img_pipe = None
_controlnet_pipe = None
_upscale_pipe = None

def get_txt2img_pipe():
    global _txt2img_pipe
    if _txt2img_pipe is None:
        print("Loading SDXL txt2img pipeline…")
        _txt2img_pipe = StableDiffusionXLPipeline.from_pretrained(
            TXT2IMG_MODEL_ID, torch_dtype=DTYPE
        )
        try:
            _txt2img_pipe.enable_xformers_memory_efficient_attention()
        except Exception as e:
            print("xFormers not enabled:", e)
        _txt2img_pipe.to(DEVICE)
    return _txt2img_pipe

def get_img2img_pipe():
    global _img2img_pipe
    if _img2img_pipe is None:
        print("Loading SDXL img2img pipeline…")
        _img2img_pipe = StableDiffusionXLImg2ImgPipeline.from_pretrained(
            TXT2IMG_MODEL_ID, torch_dtype=DTYPE
        )
        try:
            _img2img_pipe.enable_xformers_memory_efficient_attention()
        except Exception as e:
            print("xFormers not enabled:", e)
        _img2img_pipe.to(DEVICE)
    return _img2img_pipe

def get_controlnet_pipe():
    global _controlnet_pipe
    if _controlnet_pipe is None:
        print("Loading SDXL ControlNet (Canny)…")
        controlnet = ControlNetModel.from_pretrained(CONTROLNET_CANNY_ID, torch_dtype=DTYPE)
        _controlnet_pipe = StableDiffusionXLControlNetPipeline.from_pretrained(
            TXT2IMG_MODEL_ID, controlnet=controlnet, torch_dtype=DTYPE
        )
        try:
            _controlnet_pipe.enable_xformers_memory_efficient_attention()
        except Exception as e:
            print("xFormers not enabled:", e)
        _controlnet_pipe.to(DEVICE)
    return _controlnet_pipe

def get_upscale_pipe():
    global _upscale_pipe
    if _upscale_pipe is None:
        print("Loading x4 Upscaler…")
        _upscale_pipe = StableDiffusionUpscalePipeline.from_pretrained(UPSCALE_MODEL_ID, torch_dtype=DTYPE)
        try:
            _upscale_pipe.enable_xformers_memory_efficient_attention()
        except Exception as e:
            print("xFormers not enabled:", e)
        _upscale_pipe.to(DEVICE)
    return _upscale_pipe

print("Loader functions ready.")

In [ ]:
# 🎨 Color tone control (widget-style via variables)
# You can change these any time before generation
COLOR_TONE_PRESETS = [
    "warm earthy tones",
    "cool tones",
    "sepia",
    "black and white ink",
    "muted colors",
    "vivid watercolor",
]
color_tone = "warm earthy tones"  # default
custom_tone = ""  # if set non-empty, overrides color_tone
print("Current color tone:", custom_tone or color_tone)

In [ ]:
# 🧾 Style base & prompt builder
STYLE_BASE = (
    "Vintage-style illustration, hand-drawn with ink and watercolor effect on textured parchment background. "
    "Caricatured and symbolic, with {tone}. "
    "Characters in modest or historical attire, expressive faces, bold outlines, soft shading. "
    "Old book illustration style, retro cartoon aesthetic."
)

def build_prompt(scene: str, tone: str | None = None):
    tone_final = (custom_tone or tone or color_tone).strip()
    return STYLE_BASE.format(tone=tone_final) + (" " + scene if scene else "")

print(build_prompt(""))

In [ ]:
# 🖼 Text2Img generation
scene = "A thoughtful woman in modest clothing, sitting with hand on chin, pensive pose."
height, width = 1024, 1024
steps = 30
cfg_scale = 6.5
seed = 12345  # set to None for random
num_images = 1

pipe = get_txt2img_pipe()
generator = set_seed(seed)
prompt = build_prompt(scene)
print("PROMPT:\n", prompt)

images = pipe(
    prompt,
    height=height,
    width=width,
    num_inference_steps=steps,
    guidance_scale=cfg_scale,
    generator=generator,
).images

paths = []
for i, im in enumerate(images[:num_images]):
    p = save_image(im, prefix="text2img")
    paths.append(p)
print("Done.")
paths

In [ ]:
# 🖼 Img2Img (photo → vintage)
from io import BytesIO
try:
    from google.colab import files
    COLAB = True
except Exception:
    COLAB = False

input_image_path = ""  # optional: set a path like "/content/my_photo.jpg"; leave empty to upload via dialog
strength = 0.5  # 0.3–0.8 (higher = more stylized)

if not input_image_path:
    if COLAB:
        print("Upload an input image…")
        up = files.upload()
        input_image_path = list(up.keys())[0]
    else:
        raise ValueError("Please set input_image_path or run in Colab to upload.")

init_img = Image.open(input_image_path).convert("RGB")
pipe_i2i = get_img2img_pipe()
prompt = build_prompt("")  # style only; you can append scene details
print("PROMPT:\n", prompt)

generator = set_seed(seed)
res = pipe_i2i(
    prompt=prompt,
    image=init_img,
    strength=strength,
    num_inference_steps=steps,
    guidance_scale=cfg_scale,
    generator=generator,
)
out_img = res.images[0]
save_image(out_img, prefix="img2img")

In [ ]:
# 🧭 ControlNet (Canny) for consistency
def canny_from_image(pil_img: Image.Image, low=100, high=200):
    arr = np.array(pil_img.convert('RGB'))
    edges = cv2.Canny(arr, low, high)
    edges_3c = cv2.cvtColor(edges, cv2.COLOR_GRAY2RGB)
    return Image.fromarray(edges_3c)

control_input_path = ""  # optional; leave empty to upload
canny_low, canny_high = 100, 200

if not control_input_path:
    if COLAB:
        print("Upload an image for ControlNet Canny…")
        up2 = files.upload()
        control_input_path = list(up2.keys())[0]
    else:
        raise ValueError("Please set control_input_path or run in Colab to upload.")

cn_img_src = Image.open(control_input_path).convert("RGB")
cn_img = canny_from_image(cn_img_src, canny_low, canny_high)
display(cn_img)

pipe_cn = get_controlnet_pipe()
scene_cn = "Group of legislators sitting around a wooden table, engaged in serious discussion."
prompt_cn = build_prompt(scene_cn)
print("PROMPT:\n", prompt_cn)

generator = set_seed(seed)
out = pipe_cn(
    prompt=prompt_cn,
    image=cn_img,
    num_inference_steps=steps,
    guidance_scale=cfg_scale,
    generator=generator,
)
cn_out = out.images[0]
save_image(cn_out, prefix="controlnet_canny")

In [ ]:
# ⬆️ Upscale x4
up_input_path = ""  # set a path to image or leave empty to use the last saved image

def find_latest_image(folder=OUTDIR):
    imgs = [os.path.join(folder, f) for f in os.listdir(folder) if f.lower().endswith((".png",".jpg",".jpeg"))]
    if not imgs:
        return None
    return max(imgs, key=os.path.getmtime)

if not up_input_path:
    up_input_path = find_latest_image()
    assert up_input_path, "No images found to upscale. Generate something first."

img = Image.open(up_input_path).convert("RGB")
pipe_up = get_upscale_pipe()
upscaled = pipe_up(prompt="" , image=img).images[0]
save_image(upscaled, prefix="upscaled_x4")

## ✅ Tips
- Change `color_tone` or `custom_tone` any time, then regenerate.
- Use `seed=None` for randomization or set a fixed integer for repeatability.
- For Img2Img, tweak `strength`: lower = closer to original, higher = stronger style.
- ControlNet canny retains silhouette/contours. For different looks, adjust `canny_low/high`.
- If memory errors occur, run `free_memory()` and re-run only the needed loader.
